# **CS452 Project (State Farm Distracted Driver Detection)**
By Radheem Razi (K163645), Abdul Mannan (K163620), Murtaza Multanwala (K163618), Shahyar (K163750) & Saira (K163665)

## **Introduction**
This notebook is part of our Deep Learning for Visual Perception (CS452) project for Spring '20


We are trying to solve the problem of detecting distracted drivers by participating in a featured Kaggle competition called "State Farm Distracted Driver Detection".
This dataset is available on Kaggle, under the State Farm competition: https://www.kaggle.com/c/state-farm-distracted-driver-detection
This notebook mostly revolves around investigating and visualizing the given dataset.

There are 24 versions and each version has saved the results of each portion of the notebook.

## Import the Libraries


In [ ]:
import os
import glob
import random
import gc
import pickle
import time
import tensorflow
import datetime
import math
from skimage import color

os.environ['KERAS_BACKEND'] = 'tensorflow'
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # 3 = INFO, WARNING, and ERROR messages are not printed


from tqdm import tqdm

import numpy as np
import pandas as pd
from IPython.display import FileLink
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns 
%matplotlib inline

from IPython.display import display, Image
import matplotlib.image as mpimg
import cv2
import PIL
from PIL import Image

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_files       
from keras.utils import np_utils
from sklearn.utils import shuffle
from sklearn.metrics import log_loss

from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D,MaxPool2D
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19


from keras.utils import to_categorical
from keras.models import Model
from keras import optimizers

from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input

from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from tensorflow.keras.losses import CategoricalCrossentropy
from sklearn.metrics import classification_report
from keras.callbacks import LearningRateScheduler

## Import the Datasets


In [ ]:
activity_map = {'c0': 'Safe driving', 
                'c1': 'Texting - right', 
                'c2': 'Talking on the phone - right', 
                'c3': 'Texting - left', 
                'c4': 'Talking on the phone - left', 
                'c5': 'Operating the radio', 
                'c6': 'Drinking', 
                'c7': 'Reaching behind', 
                'c8': 'Hair and makeup', 
                'c9': 'Talking to passenger'}

In [ ]:
dataset = pd.read_csv('../input/state-farm-distracted-driver-detection/driver_imgs_list.csv')
dataset.head(5)


#### Import Driver Dataset

In [ ]:
by_drivers = dataset.groupby('subject')
unique_drivers = by_drivers.groups.keys()
print(unique_drivers)

In [ ]:
img_rows = 224
img_cols = 224
color_type = 3


In [ ]:
start_time = time.time()
driver_details = pd.read_csv('../input/state-farm-distracted-driver-detection/driver_imgs_list.csv',na_values='na')

train_image = []
image_label = []


for i in range(10):
    print('now we are in the folder C',i)
    imgs = os.listdir("../input/state-farm-distracted-driver-detection/imgs/train/c"+str(i))
    print(len(imgs))
    for j in range(1300):               ########comment range(len(imgs)):#1900 (DUE TO MEMORY LIMITATIONS UNDERSAMPLED DATASET HERE)
                                                                         #  (In colab notebook all training images are considered when training)                     
        
        
        img_name = "../input/state-farm-distracted-driver-detection/imgs/train/c"+str(i)+"/"+imgs[j]
        img = cv2.imread(img_name)
        
        #img = Image.open(img_name)
        #wpercent = (img_rows/float(img.size[0]))
        #hsize = int((float(img.size[1])*float(wpercent)))
        #img = img.resize((img_rows,hsize), PIL.Image.ANTIALIAS)
        #img=np.array(img)
        #img=img/255
        
        #img = color.rgb2gray(img)
        #img = img[50:,120:-50]           ########uncomment to crop
        
        img = cv2.resize(img,(img_rows,img_cols))
        label = i
        driver = driver_details[driver_details['img'] == imgs[j]]['subject'].values[0]
        train_image.append([img,label,driver])
        image_label.append(i)
if not os.path.exists('cache'):        
        os.mkdir('cache')
        
file = open('cache/train224.dat', 'wb')    ##saving cache to save time 
pickle.dump(train_image, file)
file.close()
print("--- %s seconds ---" % (time.time() - start_time))

Class imbalance problem exists hence prevented (in this notebook) by under sampling technique

In [ ]:
train_image = []
file = open('cache/train224.dat', 'rb')
train_image=pickle.load(file)
file.close()

In [ ]:
len(train_image)

In [ ]:
i = 0
fig, ax = plt.subplots(1, 2, figsize = (img_rows,img_cols ))
for i in range(2):
    #print ('Image number:',i)
    img = train_image[i][0]
    #img = color.rgb2gray(img)
    #\img = img[50:,120:-50]
    #img = cv2.resize(img,(img_rows,img_cols))
    
    ax[i].imshow(img)
    plt.show


# Train and validation split based on driver ids

**Eliminating Data leakage**

> For random drivers:

In [ ]:
# ## getting list of driver names

# D = []
# for features,labels,drivers in train_image:
#     D.append(drivers)

# ## Deduplicating drivers

# deduped = []

# for i in D:
#     if i not in deduped:
#         deduped.append(i)
    

# ## selecting random drivers for the validation set
# driv_selected = []
# import random
# driv_nums = random.sample(range(len(deduped)), 4)
# for i in driv_nums:
#     driv_selected.append(deduped[i])


> Explicitly mentioned drivers (Based on random CV)

In [ ]:
driv_selected =  ['p061', 'p035', 'p056', 'p052','p042'] # ['p061', 'p035', 'p056', 'p052'] ['p050', 'p015', 'p022', 'p056','p042']
### NEW RANDOM SEARCH ['p014', 'p012', 'p064', 'p039'] 2.27 start
X_train= []
y_train = []
X_test = []
y_test = []
D_train = []
D_test = []
true_test = []

for features,labels,drivers in train_image:
    if drivers in driv_selected:
        X_test.append(features)
        y_test.append(labels)
        D_test.append(drivers)
        true_test.append(labels)
    
    else:
        X_train.append(features)
        y_train.append(labels)
        D_train.append(drivers)
    
print (len(X_train),len(X_test))
print (len(y_train),len(y_test))

In [ ]:
#X_train = np.array(X_train).reshape(-1,img_rows,img_cols,color_type)
#X_test = np.array(X_test).reshape(-1,img_rows,img_cols,color_type)
X_train = np.array(X_train)
X_test = np.array(X_test)

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
img_rows=X_train.shape[1]
img_cols=X_train.shape[2]
print(img_rows,img_cols)

print (X_train.shape)
print(y_train.shape)

In [ ]:
del train_image,img
gc.collect()

#### Statistics

In [ ]:
# Statistics
from glob import glob

# Load the list of names
names = [item[17:19] for item in sorted(glob("../input/imgs/train/*/"))]
test_files_size = len(np.array(glob(os.path.join('..', 'input/imgs', 'test', '*.jpg'))))
x_train_size = len(X_train)
categories_size = len(names)
x_test_size = len(X_test)
print('There are %s total images.\n' % (test_files_size + x_train_size + x_test_size))
print('There are %d training images.' % x_train_size)
print('There are %d total training categories.' % categories_size)
print('There are %d validation images.' % x_test_size)
print('There are %d test images.'% test_files_size)

#### Dataset Visualization

In [ ]:
# Plot figure size
plt.figure(figsize = (10,10))
# Count the number of images per category
sns.countplot(x = 'classname', data = dataset)
# Change the Axis names
plt.ylabel('Count')
plt.title('Categories Distribution')
# Show plot
plt.show()

In [ ]:
# Find the frequency of images per driver
drivers_id = pd.DataFrame((dataset['subject'].value_counts()).reset_index())
drivers_id.columns = ['driver_id', 'Counts']
drivers_id

In [ ]:
# Plotting class distribution
dataset['class_type'] = dataset['classname'].str.extract('(\d)',expand=False).astype(np.float)
plt.figure(figsize = (20,20))
dataset.hist('class_type', alpha=0.5, layout=(1,1), bins=10)
plt.title('Class distribution')
plt.show()

# **Images belonging to different categories**

In [ ]:
activity_map = {'c0': 'Safe driving', 
                'c1': 'Texting - right', 
                'c2': 'Talking on the phone - right', 
                'c3': 'Texting - left', 
                'c4': 'Talking on the phone - left', 
                'c5': 'Operating the radio', 
                'c6': 'Drinking', 
                'c7': 'Reaching behind', 
                'c8': 'Hair and makeup', 
                'c9': 'Talking to passenger'}

In [ ]:
plt.figure(figsize = (12, 20))
image_count = 1
BASE_URL = '../input/imgs/train/'
for directory in os.listdir(BASE_URL):
    if directory[0] != '.':
        for i, file in enumerate(os.listdir(BASE_URL + directory)):
            if i == 1:
                break
            else:
                fig = plt.subplot(5, 2, image_count)
                image_count += 1
                image = mpimg.imread(BASE_URL + directory + '/' + file)
                plt.imshow(image)
                plt.title(activity_map[directory])

In [ ]:
batch_size = 4 #648
nb_epoch = 100

## **CNN model**

Training results saved in earlier version of this notebook

In [ ]:
batch_size = 32
nb_epoch = 100

In [ ]:
models_dir = "saved_models"
if not os.path.exists(models_dir):
    os.makedirs(models_dir)
    
checkpointer = ModelCheckpoint(filepath='saved_models/weights_best_vanilla.hdf5', 
                               monitor='val_loss', mode='min',
                               verbose=1, save_best_only=True)
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
callbacks = [checkpointer, es]

In [ ]:
def create_model_v2():
    # Optimised Vanilla CNN model
    model = Sequential()

    ## CNN 1
    model.add(Conv2D(32,(3,3),activation='relu',input_shape=(img_rows, img_cols, color_type)))
    model.add(BatchNormalization())
    model.add(Conv2D(32,(3,3),activation='relu',padding='same'))
    model.add(BatchNormalization(axis = 3))
    model.add(MaxPooling2D(pool_size=(2,2),padding='same'))
    model.add(Dropout(0.3))

    ## CNN 2
    model.add(Conv2D(64,(3,3),activation='relu',padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(64,(3,3),activation='relu',padding='same'))
    model.add(BatchNormalization(axis = 3))
    model.add(MaxPooling2D(pool_size=(2,2),padding='same'))
    model.add(Dropout(0.3))

    ## CNN 3
    model.add(Conv2D(128,(3,3),activation='relu',padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(128,(3,3),activation='relu',padding='same'))
    model.add(BatchNormalization(axis = 3))
    model.add(MaxPooling2D(pool_size=(2,2),padding='same'))
    model.add(Dropout(0.5))

    ## Output
    model.add(Flatten())
    model.add(Dense(512,activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(128,activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(10,activation='softmax'))

    return model

In [ ]:
model_v2 = create_model_v2()

model_v2.summary()

# Compiling the model
model_v2.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint,EarlyStopping
nb_train_samples = X_train.shape[0]
nb_validation_samples = X_test.shape[0]
print(nb_train_samples)
print(nb_validation_samples)

datagen = ImageDataGenerator(
    height_shift_range=0.5,
    width_shift_range = 0.5,
    zoom_range = 0.5,
    rotation_range=30
        )

training_generator = datagen.flow(X_train, y_train, batch_size=batch_size)
validation_generator = datagen.flow(X_test, y_test, batch_size=batch_size)

history_v3 = model_v2.fit_generator(training_generator,
                         steps_per_epoch = nb_train_samples // batch_size,
                         epochs = nb_epoch, 
                         callbacks=callbacks,
                         verbose = 1,
                         validation_data = validation_generator,
                         validation_steps = nb_validation_samples // batch_size)


In [ ]:
model_v2.load_weights('saved_models/weights_best_vanilla.hdf5')

In [ ]:
score = model_v2.evaluate(X_test, y_test, verbose=1)
print('Score: ', score)

y_pred = model_v2.predict(X_test, batch_size=batch_size, verbose=1)
score = log_loss(y_test, y_pred)
print('Score log loss:', score)

In [ ]:
score = model_v2.evaluate_generator(validation_generator, nb_validation_samples // batch_size)
print("Test Score:", score[0])
print("Test Accuracy:", score[1])

In [ ]:
def plot_test_class(model, test_files, image_number, color_type=1):
    img_brute = test_files[image_number]
    img_brute = cv2.resize(img_brute,(img_rows,img_cols))
    plt.imshow(img_brute, cmap='gray')

    new_img = img_brute.reshape(-1,img_rows,img_cols,color_type)

    y_prediction = model.predict(new_img, batch_size=batch_size, verbose=1)
    print('Y prediction: {}'.format(y_prediction))
    print('Predicted: {}'.format(activity_map.get('c{}'.format(np.argmax(y_prediction)))))
    
    plt.show()
    
plot_test_class(model_v2, test_files, 143,color_type) # The model really performs badly

##  **Transfer Learning (VGG-19)**

In [ ]:
def vgg_std19_model(img_rows, img_cols, color_type=3):
    nb_classes = 10
    # Remove fully connected layer and replace
    # with softmax for classifying 10 classes
    vgg19_model = VGG19(weights="imagenet", include_top=False)

    # Freeze all layers of the pre-trained model
    #for layer in vgg16_model.layers:
     #   layer.trainable = False
        
    x = vgg19_model.output
    x = GlobalAveragePooling2D()(x)
    #x = Dense(1024, activation='relu')(x)
    predictions = Dense(nb_classes, activation = 'softmax')(x)

    model = Model(input = vgg19_model.input, output = predictions)
    
    return model

In [ ]:
# Load the VGG19 network
print("Loading network...")
model_vgg19 = vgg_std19_model(img_rows, img_cols)

model_vgg19.summary()
sgd = optimizers.SGD(lr=0.0001,momentum=0.9) # try 0.01 - didn't converge and 0.005 , 0.001 best acc of 11%
#r=1e-4, momentum=0.9 old- lr = 0.005
#rms=optimizers.RMSprop(lr=1e-5)
cc=CategoricalCrossentropy(label_smoothing=5e-5) 
#model_vgg16.compile(loss='categorical_crossentropy', optimizer='rmsprop',   metrics=['accuracy'])
#sgd = optimizers.SGD(lr = 0.005) # try 0.01 - didn't converge and 0.005 , 0.001 best acc of 11%

model_vgg19.compile(optimizer=sgd,loss=cc,metrics=['accuracy'])

In [ ]:

datagen = ImageDataGenerator(
    height_shift_range=0.5,
    width_shift_range = 0.5,
    zoom_range = 0.5,
    rotation_range=30
        )
#datagen.fit(X_train)
data_generator = datagen.flow(X_train, y_train, batch_size = batch_size)
#mobilenet_model=model.fit(X_train, Y_train, batch_size=32, nb_epoch=25, verbose=1, validation_data=(X_valid, Y_valid))

# Fits the model on batches with real-time data augmentation:
#mobilenet_model = model.fit_generator(data_generator,steps_per_epoch = len(X_train) / 32, callbacks=[checkpointer, earlystopper],
 #                                                         epochs = nb_epoch, verbose = 1, validation_data = (X_valid, Y_valid))


nb_train_samples = X_train.shape[0]
nb_validation_samples = X_test.shape[0]

In [ ]:
# Training the vgg Model
models_dir = "saved_models"
if not os.path.exists(models_dir):
    os.makedirs(models_dir)
    
checkpoint = ModelCheckpoint('saved_models/weights_best_vgg16_model.hdf5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

history_v4 = model_vgg16.fit_generator(data_generator,
                         steps_per_epoch = nb_train_samples // batch_size,
                         epochs = nb_epoch, 
                         callbacks=[es, checkpoint],
                         verbose = 1,
                         validation_data = (X_test, y_test),
                         validation_steps = nb_validation_samples // batch_size)

In [ ]:
model_vgg19.load_weights('saved_models/weights_best_vgg19.hdf5')

# **Transfer Learning (Xception)**

In [ ]:
gc.enable()
gc.collect()

In [ ]:
def modelXception_V1():
    import keras
    from keras.layers import Input
    xception_input = Input(shape = (img_rows,img_cols, color_type), name = 'Image_input')

    ## The RESNET model

    from keras.applications.xception import preprocess_input, decode_predictions
    from keras.applications.xception import Xception


    #Get the RESNET weights and layers

    model_xception_conv = Xception(weights= 'imagenet', include_top=False, input_shape= (img_rows,img_cols, color_type))
    
    
    output_xception_conv = model_xception_conv(xception_input)

    #Add the fully-connected layers 

    x=GlobalAveragePooling2D()(output_xception_conv)
    x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
    x = Dropout(0.1)(x) # **reduce dropout 
    x=Dense(1024,activation='relu')(x) #dense layer 2
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(512,activation='relu')(x) #dense layer 3
    x = Dense(10, activation='softmax', name='predictions')(x)


    xception_pretrained = Model(input = xception_input, output = x)
    # for layer in resnet50_pretrained.layers[:2]:
    #     layer.trainable=False
    # for layer in resnet50_pretrained.layers[2:]:
    #     layer.trainable=True
    for layer in xception_pretrained.layers:
        print("{}: {}".format(layer, layer.trainable))

    xception_pretrained.summary()


    # Compile CNN model
    adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8, decay=0.0)
    sgd = optimizers.SGD(lr = 0.005)
    xception_pretrained.compile(loss='categorical_crossentropy',optimizer = sgd,metrics=['accuracy'])
    
    return xception_pretrained
    
    

In [ ]:
def modelXception()_V2:
    import keras
    from keras.layers import Input
    xception_input = Input(shape = (img_rows,img_cols, color_type), name = 'Image_input')

    ## The RESNET model

    from keras.applications.xception import preprocess_input, decode_predictions
    from keras.applications.xception import Xception


    #Get the RESNET weights and layers

    model_xception_conv = Xception(weights= 'imagenet', include_top=False, input_shape= (img_rows,img_cols, color_type))
    
    
    output_xception_conv = model_xception_conv(xception_input)

    #Add the fully-connected layers 

    x=GlobalAveragePooling2D()(output_xception_conv)
    #x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
    #x = Dropout(0.1)(x) # **reduce dropout 
    #x=Dense(1024,activation='relu')(x) #dense layer 2
    #x = BatchNormalization()(x)
    #x = Dropout(0.5)(x)
    #x = Dense(512,activation='relu')(x) #dense layer 3
    x = Dense(10, activation='softmax', name='predictions')(x)


    xception_pretrained = Model(input = xception_input, output = x)
    # for layer in resnet50_pretrained.layers[:2]:
    #     layer.trainable=False
    # for layer in resnet50_pretrained.layers[2:]:
    #     layer.trainable=True
    for layer in xception_pretrained.layers:
        print("{}: {}".format(layer, layer.trainable))

    xception_pretrained.summary()


    # Compile CNN model
    #adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8, decay=0.0)
    sgd = optimizers.SGD(lr=5e-4, momentum=0.9)#lr = 0.005)
    cc=CategoricalCrossentropy(label_smoothing=5e-5) 

    xception_pretrained.compile(loss=cc,optimizer = sgd,metrics=['accuracy'])
    
    return xception_pretrained
    


In [ ]:
modelXception=modelXception_V2()
modelXception.summary()


In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint,EarlyStopping

checkpointer = ModelCheckpoint('xception_weights_aug_extralayer_alltrained_sgd2_V2.hdf5', verbose=1, save_best_only=True)
earlystopper = EarlyStopping(monitor='val_loss', patience=15, verbose=1)


datagen = ImageDataGenerator(
    height_shift_range=0.5,
    width_shift_range = 0.5,
    zoom_range = 0.5,
    rotation_range=30,
    shear_range=0.2
        )

#datagen.fit(X_train)
data_generator = datagen.flow(X_train, y_train, batch_size = batch_size)

# Fits the model on batches with real-time data augmentation:
xception_model = model.fit_generator(data_generator,steps_per_epoch = len(X_train) / batch_size, callbacks=[checkpointer, earlystopper],
                                                            epochs = nb_epoch, verbose = 1, validation_data = (X_test, y_test))


# **Transfer Learning (ResNet)**

In [ ]:
from keras.layers import Input
resnet50_input = Input(shape = (img_rows, img_cols, 3), name = 'Image_input')

## The RESNET model

from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.applications.resnet50 import ResNet50


#Get the RESNET weights and layers

model_resnet50_conv = ResNet50(weights= 'imagenet', include_top=False, input_shape= (img_rows,img_cols,3))

In [ ]:
from keras.models import Model


x=model_resnet50_conv.output
x=GlobalAveragePooling2D()(x)

x = Dropout(0.5)(x)

x=Dense(512,activation='relu')(x)

preds=Dense(10,activation='softmax')(x) #final layer with softmax activation

resnet50_pretrained = Model(inputs=model_resnet50_conv.input, outputs=preds)
    
#output_resnet50_conv = model_resnet50_conv(resnet50_input)
#Add the fully-connected layers 
#x=GlobalAveragePooling2D()(output_resnet50_conv)
#x = Flatten(name='flatten')(output_resnet50_conv)
#x = Dense(500, activation='relu', name='fc1')(x)
#x = Dense(500, activation='relu', name='fc2')(x)
#x = Dense(10, activation='softmax', name='predictions')(x)


#resnet50_pretrained = Model(input = resnet50_input, output = x)

resnet50_pretrained.summary()

# Compile CNN model
#adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8, decay=0.0)

def custom_loss(y_true, y_pred):
    return tf.keras.losses.CategoricalCrossentropy(y_true, y_pred, label_smoothing=0.1)

def step_decay(epoch):
    initial_lrate = 0.001
    drop = 0.5
    epochs_drop = 10.0
    lrate = initial_lrate * math.pow(drop,  
        math.floor((1+epoch)/epochs_drop))
    return lrate

lrate = LearningRateScheduler(step_decay)

sgd = optimizers.SGD( lr=5e-4, momentum=0.9)#0.001)
cc=CategoricalCrossentropy(label_smoothing=5e-5)

resnet50_pretrained.compile(loss=cc,optimizer = sgd,metrics=['accuracy'])

In [ ]:
from keras.models import Model


output_resnet50_conv = model_resnet50_conv(resnet50_input)

#Add the fully-connected layers 

x = Flatten(name='flatten')(output_resnet50_conv)
x = Dense(500, activation='relu', name='fc1')(x)
x = Dense(500, activation='relu', name='fc2')(x)
x = Dense(10, activation='softmax', name='predictions')(x)


resnet50_pretrained = Model(input = resnet50_input, output = x)

resnet50_pretrained.summary()

# Compile CNN model
#adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8, decay=0.0)

def custom_loss(y_true, y_pred):
    return tf.keras.losses.CategoricalCrossentropy(y_true, y_pred, label_smoothing=0.1)

def step_decay(epoch):
    initial_lrate = 0.001
    drop = 0.5
    epochs_drop = 10.0
    lrate = initial_lrate * math.pow(drop,  
        math.floor((1+epoch)/epochs_drop))
    return lrate

lrate = LearningRateScheduler(step_decay)

sgd = optimizers.SGD( lr=5e-4, momentum=0.9)#0.001)
cc=CategoricalCrossentropy(label_smoothing=5e-5)

resnet50_pretrained.compile(loss=cc,optimizer = sgd,metrics=['accuracy'])

In [ ]:
from keras.models import Model


x=model_resnet50_conv.output
x=GlobalAveragePooling2D()(x)

#x = Dropout(0.001)(x)

#x=Dense(512,activation='relu')(x)

preds=Dense(10,activation='softmax')(x) #final layer with softmax activation

resnet50_pretrained = Model(inputs=model_resnet50_conv.input, outputs=preds)
    
#output_resnet50_conv = model_resnet50_conv(resnet50_input)
#Add the fully-connected layers 
#x=GlobalAveragePooling2D()(output_resnet50_conv)
#x = Flatten(name='flatten')(output_resnet50_conv)
#x = Dense(500, activation='relu', name='fc1')(x)
#x = Dense(500, activation='relu', name='fc2')(x)
#x = Dense(10, activation='softmax', name='predictions')(x)


#resnet50_pretrained = Model(input = resnet50_input, output = x)

resnet50_pretrained.summary()

# Compile CNN model
#adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8, decay=0.0)

def custom_loss(y_true, y_pred):
    return tf.keras.losses.CategoricalCrossentropy(y_true, y_pred, label_smoothing=0.1)

def step_decay(epoch):
    initial_lrate = 0.001
    drop = 0.5
    epochs_drop = 10.0
    lrate = initial_lrate * math.pow(drop,  
        math.floor((1+epoch)/epochs_drop))
    return lrate

lrate = LearningRateScheduler(step_decay)

sgd = optimizers.SGD( lr=5e-4, momentum=0.9)#0.001)
cc=CategoricalCrossentropy(label_smoothing=5e-5)

resnet50_pretrained.compile(loss=cc,optimizer = sgd,metrics=['accuracy'])

In [ ]:
from keras.models import Model


x=model_resnet50_conv.output
x=GlobalAveragePooling2D()(x)

x = Dropout(0.001)(x)

x=Dense(512,activation='relu')(x)

preds=Dense(10,activation='softmax')(x) #final layer with softmax activation

resnet50_pretrained = Model(inputs=model_resnet50_conv.input, outputs=preds)
    
#output_resnet50_conv = model_resnet50_conv(resnet50_input)
#Add the fully-connected layers 
#x=GlobalAveragePooling2D()(output_resnet50_conv)
#x = Flatten(name='flatten')(output_resnet50_conv)
#x = Dense(500, activation='relu', name='fc1')(x)
#x = Dense(500, activation='relu', name='fc2')(x)
#x = Dense(10, activation='softmax', name='predictions')(x)


#resnet50_pretrained = Model(input = resnet50_input, output = x)

resnet50_pretrained.summary()

# Compile CNN model
#adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8, decay=0.0)

def custom_loss(y_true, y_pred):
    return tf.keras.losses.CategoricalCrossentropy(y_true, y_pred, label_smoothing=0.1)

def step_decay(epoch):
    initial_lrate = 0.001
    drop = 0.5
    epochs_drop = 10.0
    lrate = initial_lrate * math.pow(drop,  
        math.floor((1+epoch)/epochs_drop))
    return lrate

lrate = LearningRateScheduler(step_decay)

sgd = optimizers.SGD( lr=5e-4, momentum=0.9)#0.001)
cc=CategoricalCrossentropy(label_smoothing=5e-5)

resnet50_pretrained.compile(loss=cc,optimizer = sgd,metrics=['accuracy'])

In [ ]:
from keras.callbacks import ModelCheckpoint,EarlyStopping

checkpointer = ModelCheckpoint('resnet_weights_aug_alltrained_sgd2_setval.hdf5', verbose=1, save_best_only=True)
earlystopper = EarlyStopping(monitor='accuracy', patience=7, verbose=1)


datagen = ImageDataGenerator(
    height_shift_range=0.5,
    width_shift_range = 0.5,
    zoom_range = 0.5,
    rotation_range=30
        )
#datagen.fit(X_train)
data_generator = datagen.flow(X_train, y_train, batch_size = batch_size)

# Fits the model on batches with real-time data augmentation:
resnet50_model = resnet50_pretrained.fit_generator(data_generator,steps_per_epoch = len(X_train) / batch_size, callbacks=[checkpointer, earlystopper,lrate],
                                                            epochs = 40, verbose = 1, validation_data = (X_test, y_test))

# **Transfer Learning (Mobile Net)**

In [ ]:
gc.enable()
gc.collect()

In [ ]:
def modelMobileNet():
    base_model=MobileNet(weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.

    x=base_model.output
    x=GlobalAveragePooling2D()(x)

    x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.

    x = Dropout(0.1)(x) # ****reduce dropout 
    x=Dense(1024,activation='relu')(x) #dense layer 2

    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)

    x=Dense(512,activation='relu')(x) #dense layer 3

    preds=Dense(10,activation='softmax')(x) #final layer with softmax activation

    model = Model(inputs=base_model.input, outputs=preds)
    
    from keras import optimizers  

    #adam = optimizers.Adam(lr=0.001) #tried 0.0005 - too slow and didn't converge
    sgd = optimizers.SGD(lr = 0.005) # try 0.01 - didn't converge and 0.005 , 0.001 best acc of 11%

    model.compile(optimizer=sgd,loss='categorical_crossentropy',metrics=['accuracy']) # create object
    
    return model


Fine tuned mobilenet model

In [ ]:
def modelMobileNet_v1():
    base_model=MobileNet(weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.

    x=base_model.output
    x=GlobalAveragePooling2D()(x)

    x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.

    x = Dropout(0.00001)(x) # ****reduce dropout 
    x=Dense(1024,activation='relu')(x) #dense layer 2

    x = BatchNormalization()(x)
    x = Dropout(0.01)(x)

    x=Dense(512,activation='relu')(x) #dense layer 3

    preds=Dense(10,activation='softmax')(x) #final layer with softmax activation

    model = Model(inputs=base_model.input, outputs=preds)
    
    #for layer in model.layers[:-2]:
    #         layer.trainable=False
    
    for layer in model.layers:
        print("{}: {}".format(layer, layer.trainable))
    from keras import optimizers  

    #adam = optimizers.Adam(lr=0.001) #tried 0.0005 - too slow and didn't converge
    sgd = optimizers.SGD(lr=5e-4, momentum=0.9) # try 0.01 - didn't converge and 0.005 , 0.001 best acc of 11%
    #r=1e-4, momentum=0.9 old- lr = 0.005
    #rms=optimizers.RMSprop(lr=1e-5)
    cc=CategoricalCrossentropy(label_smoothing=5e-5) #label_smoothing=0.1) #'categorical_crossentropy'
    
    model.compile(optimizer=sgd,loss=cc,metrics=['accuracy']) # create object
    
    return model


In [ ]:
def modelMobileNet_Without_extra():
    import keras

    base_model=MobileNet(weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.

    x=base_model.output
    x=GlobalAveragePooling2D()(x)

    #x = Dropout(0.001)(x)

    #x=Dense(512,activation='relu')(x)
    preds=Dense(10,activation='softmax')(x) #final layer with softmax activation

    model = Model(inputs=base_model.input, outputs=preds)
    
    from keras import optimizers  
    def step_decay(epoch):
        initial_lrate = 5e-4
        drop = 0.0001
        epochs_drop = 10.0
        lrate = initial_lrate * math.pow(drop,  
            math.floor((1+epoch)/epochs_drop))
        return lrate

    lrate = LearningRateScheduler(step_decay)

    #adam = optimizers.Adam(lr=0.001) #tried 0.0005 - too slow and didn't converge
    #adam = keras.optimizers.Adam(lr=5e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-8, decay=0.0)
    sgd = optimizers.SGD(lr=5e-4, momentum=0.9) # try 0.01 - didn't converge and 0.005 , 0.001 best acc of 11%
    cc=CategoricalCrossentropy(label_smoothing=5e-5) #'categorical_crossentropy'
    #RMSprop= keras.optimizers.RMSprop(lr=5e-4)
    
    
    model.compile(optimizer=sgd,loss=cc,metrics=['accuracy']) # create object
    
    return model


In [ ]:
#model=modelMobileNet_v1()
model=modelMobileNet_Without_extra()
model.summary()
def step_decay(epoch):
    initial_lrate = 5e-4
    drop = 0.0001
    epochs_drop = 10.0
    lrate = initial_lrate * math.pow(drop,  
        math.floor((1+epoch)/epochs_drop))
    return lrate

lrate = LearningRateScheduler(step_decay)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint,EarlyStopping

#model=modelMobileNet()

checkpointer = ModelCheckpoint('mobilenet_sgd_extra_layersft_cv.hdf5', verbose=1, save_best_only=True)
earlystopper = EarlyStopping(monitor='val_loss', patience=20, verbose=1)

datagen = ImageDataGenerator(
    
    height_shift_range=0.5,
    width_shift_range = 0.5,
    zoom_range = 0.5,
    rotation_range=10
    #shear_range=0.2
        )
#datagen.fit(X_train)
data_generator = datagen.flow(X_train, y_train, batch_size = batch_size)

# Fits the model on batches with real-time data augmentation:
mobilenet_model = model.fit_generator(data_generator,steps_per_epoch = len(X_train) / batch_size, callbacks=[checkpointer, earlystopper],
                                                            epochs = nb_epoch, verbose = 1, validation_data = (X_test, y_test))

![](https://i.imgur.com/hUHTQJe.png)

In [ ]:

del model,checkpointer,earlystopper,data_generator,mobilenet_model

# Predictions on validation

In [ ]:
#model= modelMobileNet_v1()
model=modelMobileNet_Without_extra()
model.load_weights('../input/imbalance/mobilenet_sgd_extra_layers1.hdf5')

In [ ]:

model4=resnet50_pretrained.load_weights('../input/imbalance/resnet_weights_aug_alltrained_sgd2_setval2.hdf5')

In [ ]:
predictions_valid_res = resnet50_pretrained.predict(X_test, batch_size=batch_size, verbose=1)
score = log_loss(y_test, predictions_valid_res)
print('Score log_loss: ', score)
val_pred=np.argmax(predictions_valid_res,axis=1)
val=np.argmax(y_test,axis=1)
print(classification_report(val, val_pred,target_names=activity_map))

# Predictions on test

In [ ]:
test_image = []
i = 0
fig, ax = plt.subplots(1, 20, figsize = (img_rows,img_cols))

files = os.listdir('../input/state-farm-distracted-driver-detection/imgs/test/')
nums = np.random.randint(low=1, high=len(files), size=20)
for i in range(20):
    print ('Image number:',i)
    img = cv2.imread('../input/state-farm-distracted-driver-detection/imgs/test/'+files[nums[i]])
    #img = color.rgb2gray(img)
    #img = img[50:,120:-50]
    img = cv2.resize(img,(img_rows,img_cols))
    test_image.append(img)
    ax[i].imshow(img,cmap = 'gray')
    plt.show

In [ ]:
#model=modelMobileNet()
#model= vgg_std16_model(img_rows, img_cols)
#model=model_v2
#model.load_weights('saved_models/weights_best_vanilla.hdf5')
test = []

for img in test_image:
    test.append(img)
    
#model.load_weights('mobilenet_sgd_extra_layers.hdf5') #mobilenet
#model.load_weights('saved_models/weights_best_vgg16.hdf5') #vgg16

test = np.array(test).reshape(-1,img_rows,img_cols,color_type)
prediction = model.predict(test)
prediction[0:1]

In [ ]:
tags = { "C0": "safe driving",
"C1": "texting - right",
"C2": "talking on the phone - right",
"C3": "texting - left",
"C4": "talking on the phone - left",
"C5": "operating the radio",
"C6": "drinking",
"C7": "reaching behind",
"C8": "hair and makeup",
"C9": "talking to passenger" }

i = 0
fig, ax = plt.subplots(20, 1, figsize = (img_rows,img_cols))

for i in range(20):
    ax[i].imshow(test[i].squeeze())
    #predicted_class = 'C'+str(np.where(prediction[i] == np.amax(prediction[i]))[0][0])
    #ax[i].set_title(tags[predicted_class])
    plt.show

# Generating submission csv

In [ ]:
img_rows=112
img_cols = 112
color_type = 3

In [ ]:
del test
gc.collect()

In [ ]:
%%time
test_image = []
test_id=[]
i = 0
#fig, ax = plt.subplots(1, 20, figsize = (50,50 ))

files = sorted(os.listdir('../input/state-farm-distracted-driver-detection/imgs/test/'))
print(len(files))
j=len(files)/4   #####As memory keeps getting exhausted we handled the predictions on test set iteratively
print(j)
a=int(j)+int(j)
b=a+int(j)
print(b)

for i in range(int(j),a):
    print ('Image number:',str(i)+' File: '+files[i])
    
    img = cv2.imread('../input/state-farm-distracted-driver-detection/imgs/test/'+files[i])
    #img = Image.open('../input/state-farm-distracted-driver-detection/imgs/test/'+files[i])
    #wpercent = (img_rows/float(img.size[0]))
    
    #hsize = int((float(img.size[1])*float(wpercent)))
    #img = img.resize((img_rows,hsize), PIL.Image.ANTIALIAS)
    #img=np.array(img)
    
    #img = color.rgb2gray(img)
    #img = img[50:,120:-50]
    img = cv2.resize(img,(img_rows,img_cols))
    test_image.append(img)
    test_id.append(files[i])
    #ax[i].imshow(img,cmap = 'gray')
    #plt.show

In [ ]:
i = 0
fig, ax = plt.subplots(1, 3, figsize = (img_rows,img_cols))
for i in range(3):
    #print ('Image number:',i)
    img = test[i]
    #img = color.rgb2gray(img)
    #\img = img[50:,120:-50]
    #img = cv2.resize(img,(img_rows,img_cols))
    
    ax[i].imshow(img)
    plt.show


In [ ]:
%%time
test = []

for img in test_image:
    test.append(img)
    

#model.load_weights('mobilenet_sgd_extra_layers.hdf5')

test = np.array(test).reshape(-1,img_rows,img_cols,color_type)
del test_image
gc.collect()
test.shape

Ensemble for test 

In [ ]:
%%time
model1=model.load_weights('../input/imbalance/mobilenet_sgd_extra_layers1.hdf5')
model2=modelXception.load_weights('../input/imbalance/xception_weights_aug_extralayer_alltrained_sgd2_V2.hdf5')
#model3=model_vgg16.load_weights('../input/models/weights_best_vgg16.hdf5')
#model_vgg19.load_weights('../input/vgg19model/weights_best_vgg19.hdf5')
model3=resnet50_pretrained.load_weights('../input/imbalance/resnet_weights_aug_alltrained_sgd2_setval2.hdf5')


pred1=model.predict(test)
pred2=modelXception.predict(test)
#pred3=model_vgg19.predict(test)
pred3=resnet50_pretrained.predict(test)

ensemble_prediction=(pred1+pred2*0.3+pred3)/3
len(ensemble_prediction)

In [ ]:
def create_submission(predictions, test_id, info):
    result1 = pd.DataFrame(predictions, columns=['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9'])
    result1.loc[:, 'img'] = pd.Series(test_id, index=result1.index)
    now = datetime.datetime.now()
    if not os.path.isdir('subms'):
        os.mkdir('subms')
    suffix = info + '_' + str(now.strftime("%Y-%m-%d-%H-%M"))
    sub_file = os.path.join('subms', 'submission_' + suffix + '.csv')
    result1.to_csv(sub_file, index=False)
    

info_string='ens-half4'
create_submission(ensemble_prediction, test_id, info_string)

**KNN FOR TEST**

In [ ]:
## Defining the input

from keras.layers import Input
vgg16_input = Input(shape = (img_rows,img_cols,3), name = 'Image_input')


## The VGG model

from keras.applications.vgg16 import VGG16, preprocess_input

#Get back the convolutional part of a VGG network trained on ImageNet
model_vgg16_conv = VGG16(weights='imagenet', include_top=False, input_tensor = vgg16_input)
#model_vgg16_conv.summary()
print('Model Loaded')

In [ ]:
 %%time
#7min
vgg16_features_output=model_vgg16_conv.predict(test)
vgg16_features_output.shape
knn_input=np.reshape(vgg16_features_output,(vgg16_features_output.shape[0],-1))

In [ ]:
vgg16_features_output.shape

In [ ]:
file = open('test100_knninput.dat', 'wb') #saving output
pickle.dump( knn_input, file)
file.close()

In [ ]:
%%time
from sklearn.neighbors import NearestNeighbors #KNN ALGORITHM TEST (FINAL K-D TREES IN COLAB NOTEBOOK)
nbrs = NearestNeighbors(n_neighbors=10).fit(knn_input)
print("Done")

In [ ]:
gc.collect()

In [ ]:
%%time
distances, indices = nbrs.kneighbors(knn_input)
len(indices)#56 min


In [ ]:
indices[0]


In [ ]:
file = open('test32_indices.dat', 'wb')
pickle.dump( [distances, indices], file)
file.close()

In [ ]:
distancess,indices=test_image

In [ ]:
indices[0]  #[    0, 10535, 56026, 55935, 28105, 44194, 57421, 68745, 46932,50244])

In [ ]:
fig, ax = plt.subplots(10, 1, figsize = (56,56))

img = 0

for i in indices[0]:  
    ax[img].imshow(test[i])
    plt.show
    img = img+1
    

In [ ]:
%%time
knn_predictions = []
predictions = []

print(indices.shape[0])
j=indices.shape[0]/5
print(j)
a=int(j)+int(j)
b=a+int(j)
c=b+int(j)
print(b)
print(c)


for i in range(0,20):
    print(str(0),str(int(j)))
    mean_prediction = []
    for j in range(len(ensem[0])-1):
        print(str(len(ensem[0])-1))
        k=0
        for img in indices[i]:
            print("Image Number: "+str(img))
            print('')
               
            ensemble_prediction=ensem[img][:10]
            print( ensem[img][10],img)
                    
            print(ensemble_prediction[j])
            predictions.append(ensemble_prediction[j])
            k=k+1
            if k>=9:
                break
        print("herrer")
        trimmed_value = (sum(predictions) - max(predictions) - min(predictions))/(len(predictions) -2 )
        mean_value = np.mean(predictions)
        predictions = []
        
    
        mean_prediction.append(trimmed_value)
    
    
    mean_prediction = np.asarray(mean_prediction)/sum(mean_prediction)
    print("mean calculated")
    knn_predictions.append(mean_prediction)
    print("knn calculated")
    
knn_predictions = np.asarray(knn_predictions)           
        

        
print('The code is done')
knn_class = []

for i in range(len(knn_predictions)):
    knn_class.append(np.where(knn_predictions[i] == np.amax(knn_predictions[i]))[0][0])


In [ ]:
#prediction=knn_predictions
prediction=knn_predictions

tags = { "C0": "safe driving",
"C1": "texting - right",
"C2": "talking on the phone - right",
"C3": "texting - left",
"C4": "talking on the phone - left",
"C5": "operating the radio",
"C6": "drinking",
"C7": "reaching behind",
"C8": "hair and makeup",
"C9": "talking to passenger" }

i = 0
fig, ax = plt.subplots(20, 1, figsize = (img_rows,img_cols))

for i in range(20):
    ax[i].imshow(test[i].squeeze())
    predicted_class = 'C'+str(np.where(prediction[i] == np.amax(prediction[i]))[0][0])
    ax[i].set_title(tags[predicted_class])
    plt.show

In [ ]:
test = []

for img in test_image:
    test.append(img)
    

#model.load_weights('mobilenet_sgd_extra_layers.hdf5')

test = np.array(test).reshape(-1,img_rows,img_cols,color_type)
prediction = model.predict(test)
prediction[0:1]

In [ ]:
model.load_weights('../input/others/mobilenet_sgd_extra_layersft-0.197.hdf5')
pred0=model.predict(X_test)

In [ ]:
#%%time
model1=model.load_weights('../input/imbalance/mobilenet_sgd_extra_layers1.hdf5')
model2=modelXception.load_weights('../input/imbalance/xception_weights_aug_extralayer_alltrained_sgd2_V2.hdf5')
#model3=model_vgg16.load_weights('../input/models/weights_best_vgg16.hdf5')
model_vgg19.load_weights('../input/vgg19model/weights_best_vgg19.hdf5')
model3=resnet50_pretrained.load_weights('../input/imbalance/resnet_weights_aug_alltrained_sgd2_setval_1.hdf5')


pred5=model.predict(X_test)
pred6=modelXception.predict(X_test)
pred7=model_vgg19.predict(X_test)
pred8=resnet50_pretrained.predict(X_test)

#finalpred=(pred1*0.6+pred4*0.9+pred2*0.3+pred3*0.3)/4
#finalpred=(pred1*0.7+pred4*0.9)/2
finalpred=(pred5+pred6*0.3+pred8)/3
#finalpred=(pred1+pred2+pred3)/3


In [ ]:
model1=model.load_weights('../input/mobile/mobilenet_sgd_extra_layers (1).hdf5')
model2=modelXception.load_weights('../input/modelss/xception_weights_aug_extralayer_alltrained_sgd2_V2.hdf5')
#model3=model_vgg16.load_weights('../input/models/weights_best_vgg16.hdf5')
model_vgg19.load_weights('../input/vgg19model/weights_best_vgg19.hdf5')
model4=resnet50_pretrained.load_weights('../input/resnet/resnet_weights_aug_alltrained_sgd2_setval.hdf5')


pred1=model.predict(X_test)
pred2=modelXception.predict(X_test)
pred3=model_vgg19.predict(X_test)
pred4=resnet50_pretrained.predict(X_test)

prediction=(pred1*0.6+pred4*0.9+pred2*0.3+pred3*0.3)/4

In [ ]:
#predictions=(pred0*0.9+pred1*0.4+pred4*0.9+pred2*0.05+pred3*0.05)/5

pred=ensemble_predictions
#pred=(pred5+pred6*0.05+pred8*0.7)/3

#pred=(pred0*0.9+pred1*0.4+pred4*0.9+pred2*0.05+pred3*0.05+pred5+pred6*0.05+pred8*0.7)/8

score = log_loss(y_test, pred)
print('Score log_loss: ', score)
val_pred=np.argmax(pred,axis=1)

val=np.argmax(y_test,axis=1)
print(classification_report(val, val_pred,target_names=activity_map))

# Network ensemble predictions on test set

In [ ]:
from statistics import mean,median

predictions = []
ensemble_predictions = []

for i in range(len(X_test)):
#for i in range(1):
    mean_prediction = []
    
    for j in range(10):
        predictions.append(pred0[i][j]*0.9)
        predictions.append(pred1[i][j]*0.4)
        predictions.append(pred2[i][j]*0.05)
        predictions.append(pred3[i][j]*0.05)
        predictions.append(pred4[i][j]*0.9)
        
        predictions.append(pred5[i][j])
        predictions.append(pred6[i][j]*0.05)
        predictions.append(pred8[i][j]*0.7)
       
        
        trimmed_value = (sum(predictions) - max(predictions) - min(predictions))/(len(predictions) -2 )
        
        mean_value =np.mean(predictions)

        predictions = []
        mean_prediction.append(trimmed_value)
        #mean_prediction.append(mean_value)
    
    mean_prediction = mean_prediction/ sum(mean_prediction)
    ensemble_predictions.append(mean_prediction)
        
    
#ensemble_predictions = np.asarray(ensemble_predictions)        
print('Got predictions from ensemble')

* For ensemble results

In [ ]:
model1=model.load_weights('../input/mobile/mobilenet_sgd_extra_layers (1).hdf5')
model2=modelXception.load_weights('../input/modelss/xception_weights_aug_extralayer_alltrained_sgd2_V2.hdf5')
#model3=model_vgg16.load_weights('../input/models/weights_best_vgg16.hdf5')
model_vgg16.load_weights('../input/vgg19model/weights_best_vgg19.hdf5')
model4=resnet50_pretrained.load_weights('../input/resnet/resnet_weights_aug_alltrained_sgd2_setval.hdf5')



test = []

for img in test_image:
    test.append(img)
    

#model.load_weights('mobilenet_sgd_extra_layers.hdf5')

test = np.array(test).reshape(-1,img_rows,img_cols,color_type)
pred1=model.predict(test)
pred2=modelXception.predict(test)
pred3=model_vgg19.predict(test)
pred4=resnet50_pretrained.predict(test)

prediction=(pred1*0.6+pred4*0.9+pred2*0.3+pred3*0.3)/4
prediction[0:1]

In [ ]:
def create_submission(predictions, test_id, info):
    result1 = pd.DataFrame(predictions, columns=['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9'])
    result1.loc[:, 'img'] = pd.Series(test_id, index=result1.index)
    now = datetime.datetime.now()
    if not os.path.isdir('subms'):
        os.mkdir('subms')
    suffix = info + '_' + str(now.strftime("%Y-%m-%d-%H-%M"))
    sub_file = os.path.join('subms', 'submission_' + suffix + '.csv')
    result1.to_csv(sub_file, index=False)
    

info_string='ens-half4'
create_submission(prediction, test_id, info_string)

In [ ]:
del prediction,test,img,test_image,test_id

In [ ]:
gc.enable()
gc.collect()